In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 https://develo

In [2]:
#postgres connect
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-30 01:16:38--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-11-30 01:16:38 (10.4 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
#spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
#read data in
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Luggage_v1_00.tsv.gz"), sep="\t", header=True)

df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [5]:
df.count()

348657

In [6]:
#remove duplicates
df = df.dropna(how='any')
df = df.dropDuplicates()
df.count()

348613

In [7]:
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'string'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'string'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'string'),
 ('total_votes', 'string'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [8]:
#conversion string to int
from pyspark.sql.types import IntegerType
df = df.withColumn("customer_id", df["customer_id"].cast(IntegerType()))
df = df.withColumn("product_parent", df["product_parent"].cast(IntegerType()))
df = df.withColumn("star_rating", df["star_rating"].cast(IntegerType()))
df = df.withColumn("helpful_votes", df["helpful_votes"].cast(IntegerType()))
df = df.withColumn("total_votes", df["total_votes"].cast(IntegerType()))
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [9]:
#df vine
df_vine = df.select([ "review_id"
                          , "star_rating"
                          , "helpful_votes"
                          , "total_votes"
                          , "vine"
                          ]).dropDuplicates(["review_id"])
df_vine.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R106DW9WYK7M66|          5|            0|          1|   N|
|R10FHG21AGZ6U8|          5|            0|          0|   N|
|R10HRFSB6G50NA|          4|            4|          4|   N|
|R10HSOKR22BKJS|          5|            0|          3|   N|
|R10MLWE255UJPD|          1|            0|          0|   N|
|R10NTD447C4WJX|          3|            0|          0|   N|
|R10Q1WVRHD1C56|          4|            0|          0|   N|
|R10QU651K92G38|          1|            0|          0|   N|
|R10RNAP34JZ362|          5|            0|          0|   N|
|R10SR63YRN4C5Z|          5|            0|          0|   N|
|R10XIYAEA1LWSM|          2|            1|          3|   N|
|R10XN95VW4NNRA|          5|            2|          2|   N|
|R10YOCBJK1EIHQ|          5|            0|          0|   N|
|R114QWZ9S6I50V|          4|            

In [10]:
df_vine.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

In [11]:
df_vine.count()

348613

In [12]:
#df paid-vine
paid_vine = df_vine.filter('vine = "Y"')
paid_vine.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3HJAGVD491IFV|          5|          105|        105|   Y|
|R2DTP0OYTGGL3W|          4|            1|          2|   Y|
|R3UG3J2JYSYUDK|          5|            0|          0|   Y|
| RBUGDTIU9IREG|          5|            1|          3|   Y|
|R15LE4JNH0NSRX|          3|            3|          6|   Y|
|R2Y0EL2WGKEVSH|          5|            0|          0|   Y|
|R14568FBGOPC5R|          4|            0|          0|   Y|
| RFUKDYASLHD3F|          4|            1|          1|   Y|
| RWEF5ZC4G39VW|          5|            1|          1|   Y|
| RZ0JEVQ0RZMC4|          5|            3|          3|   Y|
|R13RAWMKH88CFZ|          4|            0|          0|   Y|
|R1TFHLHKIOWIN1|          4|            2|          3|   Y|
|R1UF27O45BX658|          4|            7|         10|   Y|
|R378M4MLN7OKKZ|          5|            

In [14]:
paid_vine.count()

904

In [13]:
#df not paid-vine
unpaid_vine = df_vine.filter('vine = "N"')
unpaid_vine.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R106DW9WYK7M66|          5|            0|          1|   N|
|R10FHG21AGZ6U8|          5|            0|          0|   N|
|R10HRFSB6G50NA|          4|            4|          4|   N|
|R10HSOKR22BKJS|          5|            0|          3|   N|
|R10MLWE255UJPD|          1|            0|          0|   N|
|R10NTD447C4WJX|          3|            0|          0|   N|
|R10Q1WVRHD1C56|          4|            0|          0|   N|
|R10QU651K92G38|          1|            0|          0|   N|
|R10RNAP34JZ362|          5|            0|          0|   N|
|R10SR63YRN4C5Z|          5|            0|          0|   N|
|R10XIYAEA1LWSM|          2|            1|          3|   N|
|R10XN95VW4NNRA|          5|            2|          2|   N|
|R10YOCBJK1EIHQ|          5|            0|          0|   N|
|R114QWZ9S6I50V|          4|            

In [15]:
unpaid_vine.count()

347709

In [16]:
#vine paid with rating = 5
paid_vine_rating_5 = paid_vine.filter(paid_vine["star_rating"] == 5)
paid_vine_rating_5.count()

472

In [17]:
#vine unpaid with rating = 5
unpaid_vine_rating_5 = unpaid_vine.filter(unpaid_vine["star_rating"] == 5)
unpaid_vine_rating_5.count()

216003

In [18]:
#avg
df_avg = df_vine.groupBy("vine").avg()
df_avg.select("vine", "avg(star_rating)", "avg(helpful_votes)", "avg(total_votes)").show()

+----+------------------+------------------+------------------+
|vine|  avg(star_rating)|avg(helpful_votes)|  avg(total_votes)|
+----+------------------+------------------+------------------+
|   Y|4.3584070796460175|2.3584070796460175| 2.870575221238938|
|   N|  4.22309172325134|2.0626299578095475|2.4305007923292177|
+----+------------------+------------------+------------------+



In [20]:
#count
df_vine.createOrReplaceTempView("COUNT_VINE")
spark.sql("SELECT vine, COUNT(vine) FROM COUNT_VINE GROUP BY vine").show()

+----+-----------+
|vine|count(vine)|
+----+-----------+
|   Y|        904|
|   N|     347709|
+----+-----------+



In [ ]:
#count paid vs unpaid
spark.sql("SELECT vine, COUNT(vine) FROM COUNT_VINE WHERE star_rating = 5 GROUP BY vine").show()

Paid vine reviews are a lot lower than unpaid vine reviews.  This indicates reviewers are willing to put in effort to review the product without incentive.  This indicates the program is working since people review without incentives. For paid and unpaid vines, the average star rating was high (greater than 4 stars).  The ratings compared to votes indicate that this program might not be doing as well as indicated since votes are very low despite having high ratings.


